# Indian College Data Cleaning Assignment - SOLUTIONS

**FOR INSTRUCTOR USE ONLY**

This notebook contains the solutions to all 10 questions in the data cleaning assignment.

---

## Load the Dataset

In [ ]:
import pandas as pd
import numpy as np

# Load the messy dataset
df = pd.read_csv('../Datasets/indian_college_messy_data.csv')

print(f"Dataset shape: {df.shape}")
print("\nFirst 10 rows:")
display(df.head(10))

---
# SOLUTIONS
---

## Question 1 Solution: Clean Student Names

In [ ]:
# Solution
df['name'] = df['name'].apply(lambda x: x.title())

# Alternative solution using str accessor
# df['name'] = df['name'].str.title()

print("Sample cleaned names:")
print(df['name'].head(10))

## Question 2 Solution: Clean and Standardize Email Addresses

In [ ]:
# Solution
df['email'] = df['email'].apply(lambda x: x.lower().strip() if pd.notna(x) else x)

# Alternative solution
# df['email'] = df['email'].apply(lambda x: x.lower().strip() if isinstance(x, str) else x)

print("Sample cleaned emails:")
print(df['email'].head(10))

## Question 3 Solution: Extract Clean Phone Numbers

In [ ]:
# Solution 1: Using string replacement
def clean_phone(phone):
    if pd.isna(phone):
        return phone
    # Convert to string and remove all non-digit characters
    phone_str = str(phone)
    digits = ''.join(filter(str.isdigit, phone_str))
    # Take last 10 digits (removes country code if present)
    return digits[-10:] if len(digits) >= 10 else digits

df['phone'] = df['phone'].apply(clean_phone)

# Alternative solution using replace
# df['phone'] = df['phone'].apply(lambda x: ''.join(filter(str.isdigit, str(x)))[-10:] if pd.notna(x) else x)

print("Sample cleaned phone numbers:")
print(df['phone'].head(10))
print(f"\nAll phone numbers have 10 digits: {df['phone'].dropna().apply(lambda x: len(str(x)) == 10).all()}")

## Question 4 Solution: Fix Age Column

In [ ]:
# Solution
def clean_age(age):
    # Convert to integer
    age = int(float(age))
    # Take absolute value if negative
    age = abs(age)
    # If age > 40, it was increased by 50, so subtract 50
    if age > 40:
        age = age - 50
    return age

df['age'] = df['age'].apply(clean_age)

# Alternative solution (one-liner)
# df['age'] = df['age'].apply(lambda x: abs(int(float(x))) - 50 if abs(int(float(x))) > 40 else abs(int(float(x))))

print("Age statistics after cleaning:")
print(df['age'].describe())
print(f"\nAll ages are integers: {df['age'].dtype in ['int64', 'int32']}")
print(f"Age range: {df['age'].min()} to {df['age'].max()}")

## Question 5 Solution: Standardize Gender Values

In [ ]:
# Solution
def standardize_gender(gender):
    gender_upper = str(gender).upper()
    if gender_upper in ['MALE', 'M', 'MAN']:
        return 'Male'
    elif gender_upper in ['FEMALE', 'F', 'WOMAN']:
        return 'Female'
    else:
        return 'Other'

df['gender'] = df['gender'].apply(standardize_gender)

# Alternative solution using dictionary
# gender_map = {'MALE': 'Male', 'M': 'Male', 'MAN': 'Male',
#               'FEMALE': 'Female', 'F': 'Female', 'WOMAN': 'Female',
#               'OTHER': 'Other', 'O': 'Other', 'NON-BINARY': 'Other'}
# df['gender'] = df['gender'].apply(lambda x: gender_map.get(str(x).upper(), 'Other'))

print("Gender value counts after standardization:")
print(df['gender'].value_counts())

## Question 6 Solution: Clean CGPA Values

In [ ]:
# Solution
def clean_cgpa(cgpa):
    if pd.isna(cgpa):
        return cgpa
    # Convert to float
    cgpa = float(cgpa)
    # Take absolute value
    cgpa = abs(cgpa)
    # If > 10, subtract 5
    if cgpa > 10:
        cgpa = cgpa - 5
    # Round to 2 decimal places
    return round(cgpa, 2)

df['cgpa'] = df['cgpa'].apply(clean_cgpa)

# Alternative solution (one-liner)
# df['cgpa'] = df['cgpa'].apply(lambda x: round(abs(float(x)) - 5, 2) if pd.notna(x) and abs(float(x)) > 10 else (round(abs(float(x)), 2) if pd.notna(x) else x))

print("CGPA statistics after cleaning:")
print(df['cgpa'].describe())
print(f"\nCGPA data type: {df['cgpa'].dtype}")
print(f"Valid CGPA range (5.0 to 10.0): {df['cgpa'].dropna().min()} to {df['cgpa'].dropna().max()}")

## Question 7 Solution: Clean Attendance Percentage

In [ ]:
# Solution
def clean_attendance(att):
    if pd.isna(att):
        return att
    # Convert to string and remove % if present
    att_str = str(att).replace('%', '').strip()
    # Convert to float
    att_value = float(att_str)
    # Cap at 100
    if att_value > 100:
        att_value = 100.0
    # Round to 1 decimal
    return round(att_value, 1)

df['attendance'] = df['attendance'].apply(clean_attendance)

# Alternative solution
# df['attendance'] = df['attendance'].apply(lambda x: min(round(float(str(x).replace('%', '').strip()), 1), 100.0) if pd.notna(x) else x)

print("Attendance statistics after cleaning:")
print(df['attendance'].describe())
print(f"\nMax attendance: {df['attendance'].max()}%")

## Question 8 Solution: Expand State Abbreviations

In [ ]:
# State mapping dictionary
state_map = {
    'MH': 'Maharashtra', 'KA': 'Karnataka', 'TN': 'Tamil Nadu',
    'DL': 'Delhi', 'GJ': 'Gujarat', 'WB': 'West Bengal',
    'TS': 'Telangana', 'RJ': 'Rajasthan', 'UP': 'Uttar Pradesh',
    'KL': 'Kerala'
}

# Solution
df['state'] = df['state'].apply(lambda x: state_map.get(x, x))

# Alternative solution using replace
# df['state'] = df['state'].replace(state_map)

print("State value counts after expansion:")
print(df['state'].value_counts())
print(f"\nNo abbreviations remaining: {not any(df['state'].str.len() == 2)}")

## Question 9 Solution: Standardize Boolean Columns

In [ ]:
# Solution
def to_boolean(value):
    # Convert to string and uppercase
    val_str = str(value).upper().strip()
    # Check for true values
    if val_str in ['YES', 'Y', 'TRUE', '1', 'T']:
        return True
    else:
        return False

df['fees_paid'] = df['fees_paid'].apply(to_boolean)
df['hostel'] = df['hostel'].apply(to_boolean)

# Alternative solution (one-liner)
# df['fees_paid'] = df['fees_paid'].apply(lambda x: str(x).upper().strip() in ['YES', 'Y', 'TRUE', '1', 'T'])
# df['hostel'] = df['hostel'].apply(lambda x: str(x).upper().strip() in ['YES', 'Y', 'TRUE', '1', 'T'])

print("Fees Paid value counts:")
print(df['fees_paid'].value_counts())
print("\nHostel value counts:")
print(df['hostel'].value_counts())
print(f"\nFees_paid is boolean: {df['fees_paid'].dtype == 'bool'}")
print(f"Hostel is boolean: {df['hostel'].dtype == 'bool'}")

## Question 10 Solution: Create Grade Category from CGPA

In [ ]:
# Solution
def get_grade_category(cgpa):
    if pd.isna(cgpa):
        return 'Not Available'
    elif cgpa >= 9.0:
        return 'Excellent'
    elif cgpa >= 8.0:
        return 'Very Good'
    elif cgpa >= 7.0:
        return 'Good'
    elif cgpa >= 6.0:
        return 'Average'
    else:
        return 'Below Average'

df['grade_category'] = df['cgpa'].apply(get_grade_category)

# Alternative solution using nested ternary operators
# df['grade_category'] = df['cgpa'].apply(lambda x: 'Not Available' if pd.isna(x) else 
#                                          'Excellent' if x >= 9.0 else 
#                                          'Very Good' if x >= 8.0 else 
#                                          'Good' if x >= 7.0 else 
#                                          'Average' if x >= 6.0 else 
#                                          'Below Average')

print("Grade Category distribution:")
print(df['grade_category'].value_counts())
print("\nSample data with CGPA and Grade Category:")
display(df[['name', 'cgpa', 'grade_category']].head(20))

## Bonus Solution: Remove Duplicates

In [ ]:
# Check for duplicates
print(f"Total records before removing duplicates: {len(df)}")
print(f"Duplicate records: {df.duplicated().sum()}")

# Remove duplicates
df = df.drop_duplicates()

# Alternative: Keep first occurrence
# df = df.drop_duplicates(keep='first')

# Reset index after dropping duplicates
df = df.reset_index(drop=True)

print(f"\nTotal records after removing duplicates: {len(df)}")

## Save Cleaned Dataset

In [ ]:
# Save cleaned dataset
df.to_csv('../Datasets/indian_college_cleaned_data.csv', index=False)

print("Cleaned dataset saved successfully!")
print(f"\nFinal dataset shape: {df.shape}")
print("\nData types after cleaning:")
print(df.dtypes)
print("\nMissing values after cleaning:")
print(df.isnull().sum())
print("\nFirst 10 rows of cleaned data:")
display(df.head(10))

## Summary Statistics

In [ ]:
print("=" * 80)
print("SUMMARY STATISTICS - CLEANED DATASET")
print("=" * 80)

print("\n1. Numerical Columns:")
display(df[['age', 'semester', 'cgpa', 'attendance']].describe())

print("\n2. Gender Distribution:")
print(df['gender'].value_counts())

print("\n3. Course Distribution:")
print(df['course'].value_counts())

print("\n4. Department Distribution:")
print(df['department'].value_counts())

print("\n5. State Distribution:")
print(df['state'].value_counts())

print("\n6. Fees Payment Status:")
print(df['fees_paid'].value_counts())

print("\n7. Hostel Status:")
print(df['hostel'].value_counts())

print("\n8. Grade Categories:")
print(df['grade_category'].value_counts())

---
## Additional Teaching Points

### Key Concepts Covered:

1. **Lambda Functions**: Anonymous functions used with apply()
2. **Conditional Logic**: Using if-else inside apply()
3. **String Methods**: .upper(), .lower(), .strip(), .title(), .replace()
4. **Type Conversion**: int(), float(), str()
5. **Null Handling**: pd.isna(), pd.notna()
6. **Dictionary Mapping**: Using .get() for safe lookups
7. **Data Validation**: Checking ranges and constraints
8. **Boolean Conversion**: Standardizing Yes/No values

### Common Student Mistakes to Watch For:

1. Not handling NaN values before processing
2. Forgetting to convert types before operations
3. Not stripping whitespace from strings
4. Missing edge cases (negative values, out of range)
5. Not validating results after cleaning

### Extension Activities:

1. Add more complex validation rules
2. Create custom validation functions
3. Generate data quality reports
4. Implement error logging during cleaning
5. Create before/after comparison visualizations